# Limpieza de datos

**Principal inconveniente:** Los codigos del proyecto anterior actualmente no se pueden utilizar para esta encuesta. Se deben realizar un codigo adaptado el formato de la nueva encuesta.

## Tareas de limpieza:

### Para cada tabla por separado

1. Revisar tipo de pregunta (opcion unica o multiple) 
1. Revisar si la pregunta tiene opcion de "otro" / detalle
1. Si es opcion multiple considerar las columnas adicionales 
1. Revisar si la pregunta tiene un criterio de relevancia
1. Si tiene criterio de relevancia colocar "No valido" donde corresponda (Reducir valores perdidos).
1. Revisar categorias una por una y limpiar las respuestas posibles por ejemplo: "papa", "Papa", --> "Papa" 

### Especificamente para tablas secundarias 

- Agrupar las preguntas de las tablas secundarias en columnas para la tabla principial (segun lo coordinado con Renata).
- Agregar "filas" secundaria como nuevas columnas (pregunta_A_1, pregunta_A_2, ...) para cada observacion de la tabla principal 

### Extra

- Ahora si con los datos limpios se pueden realizar los reportes preliminares para compartir con el equipo de investigacion. 

### Tiempo estimado: 

- 14 (1/13) tablas a 2 o 3 tablas/dia = 7 o 5 dias

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.notebook import tqdm
from pandas_profiling import ProfileReport

In [3]:
%matplotlib inline

In [4]:
tqdm.pandas()

## Read Data

In [5]:
DATA_FP = "inputs/encuesta_principal/Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx"

In [6]:
df_main = pd.read_excel(DATA_FP, sheet_name=0) # Main sheet
df_comp_hogar = pd.read_excel(DATA_FP, sheet_name="composicion_hogar")
df_med_control_plagas = pd.read_excel(DATA_FP, sheet_name="_5_18_medidas_control_plagas")
df_prod_control_plagas = pd.read_excel(DATA_FP, sheet_name="_5_19_productos_control_plagas")
df_control_maleza = pd.read_excel(DATA_FP, sheet_name="_5_20_control_malezas")
df_oferta_alim_prod = pd.read_excel(DATA_FP, sheet_name="_7_1_Oferta_Alimentos_Produccio")
df_productor_dejo_prod_vend = pd.read_excel(DATA_FP, sheet_name="_7_2_produc_dejo_producir_vend")
df_productos_vende_canal = pd.read_excel(DATA_FP, sheet_name="Productos_Vende_Canal")
df_productos_compra_canal = pd.read_excel(DATA_FP, sheet_name="Productos_Compra_Canal")
df_canales_venta_productores = pd.read_excel(DATA_FP, sheet_name="canales_venta_productores")
df_aplic_bioprep = pd.read_excel(DATA_FP, sheet_name="aplicacion_biopreparados")
df_per_trab_puesto_venta = pd.read_excel(DATA_FP, sheet_name="personas_trabajan_puesto_venta")
df_canales_compra_comerciantes = pd.read_excel(DATA_FP, sheet_name="canales_compra_comerciantes")
df_huerto_rel = pd.read_excel(DATA_FP, sheet_name="HUERTO_RELACIONES")

In [7]:
DATA_APOYO_FP = "inputs/encuesta_principal/Datos_Apoyos_Cat_Vec_Alim_2022_12_04.xlsx"

In [8]:
df_apoyo_recibido = pd.read_excel(DATA_APOYO_FP, sheet_name="APOYO RECIBIDO")
df_apoyo_brindado = pd.read_excel(DATA_APOYO_FP, sheet_name="APOYO BRINDADO")

In [9]:
dfs = {
    "Principal": df_main,
    "composicion_hogar": df_comp_hogar,
    "_5_18_medidas_control_plagas": df_med_control_plagas,
    "_5_19_productos_control_plagas": df_prod_control_plagas,
    "_5_20_control_malezas": df_control_maleza,
    "_7_1_Oferta_Alimentos_Produccio": df_oferta_alim_prod,
    "_7_2_produc_dejo_producir_vend": df_productor_dejo_prod_vend,
    "Productos_Vende_Canal": df_productos_vende_canal,
    "canales_venta_productores": df_canales_venta_productores,
    "aplicacion_biopreparados": df_aplic_bioprep,
    "personas_trabajan_puesto_venta": df_per_trab_puesto_venta,
    "canales_compra_comerciantes": df_canales_compra_comerciantes,
    "Productos_Compra_Canal": df_productos_compra_canal,
    "APOYO RECIBIDO": df_apoyo_recibido,
    "APOYO BRINDADO": df_apoyo_brindado,
    "HUERTO_RELACIONES": df_huerto_rel
}

## Generacion de reportes de la tabla principal

Los reportes seran divididos por Ciudad / Vecindario / Perfil y Perfil

In [10]:
for col in ["_1_4_1_ciudad", "_1_4_2_vecindario", "_1_5_perfil_entrevistado"]:
    print(f"Valores unicos de {col}")
    print(df_main[col].value_counts())
    print("-"*30)

Valores unicos de _1_4_1_ciudad
lima     492
quito    400
Name: _1_4_1_ciudad, dtype: int64
------------------------------
Valores unicos de _1_4_2_vecindario
carabayllo      273
solanda         217
pachacamac      213
la_magdalena    159
fuera            32
Name: _1_4_2_vecindario, dtype: int64
------------------------------
Valores unicos de _1_5_perfil_entrevistado
consumidor     656
productor      149
comerciante     90
Name: _1_5_perfil_entrevistado, dtype: int64
------------------------------


## Datos Generales del Cuestionario

In [11]:
df_main.columns[5:]

Index(['_1_3_nombre_encuestador', '_1_4_1_ciudad', '_1_4_2_vecindario',
       'sector_barrio_huerto', '_1_5_perfil_entrevistado', 'zona_carabayllo',
       'otra_zona_carabayllo', 'zona_pachacamac', 'otra_zona_pachacamac',
       'mercado_carabayllo',
       ...
       '_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude',
       '_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision', '_id', '_uuid',
       '_submission_time', '_validation_status', '_notes', '_status',
       '_submitted_by', '_tags'],
      dtype='object', length=820)

In [12]:
df_main[['zona_carabayllo',
       'otra_zona_carabayllo', 'zona_pachacamac', 'otra_zona_pachacamac',
       'mercado_carabayllo',]].isna().sum()

zona_carabayllo         718
otra_zona_carabayllo    816
zona_pachacamac         724
otra_zona_pachacamac    872
mercado_carabayllo      861
dtype: int64

In [13]:
df_main.columns[:40]

Index(['_index', 'start', 'end', '_1_1_codigo_cuestionario',
       '_1_2_fecha_encuesta', '_1_3_nombre_encuestador', '_1_4_1_ciudad',
       '_1_4_2_vecindario', 'sector_barrio_huerto', '_1_5_perfil_entrevistado',
       'zona_carabayllo', 'otra_zona_carabayllo', 'zona_pachacamac',
       'otra_zona_pachacamac', 'mercado_carabayllo', 'otro_mercado_carabayllo',
       'mercado_pachacamac', 'otro_mercado_pachacamac',
       '_1_6_ambito_territorial', '_1_7_1_provincia_huerto',
       '_1_7_2_canton_huerto', '_1_7_3_parroquia_huerto',
       '_2_1_consentimiento_informado', '_3_1_Nombre', 'sexo', 'edad',
       'aporta_economicamente_hogar', '_3_2_Informo_telefono',
       'numero_telefono', 'contacto_dirigente', 'contacto_administrador',
       '_3_3_1_Distrito_Canton', '_3_3_2_Provincia', '_3_3_3_Pais',
       'otro_pais', '_3_4_tiempo_vive_hogar', '_3_4_Vive_vecindario',
       '_3_6_nivel_estudio', 'nivel_completo', 'anos_nivel_incompleto'],
      dtype='object')

In [14]:
df_main[['zona_carabayllo', 'otra_zona_carabayllo']].dropna(how='all').isna().sum()

zona_carabayllo          0
otra_zona_carabayllo    98
dtype: int64

In [15]:
df_main['_1_4_2_vecindario'].value_counts()['carabayllo']

273

In [16]:
df_main['zona_carabayllo'].value_counts()

otra            80
Tungasuca       61
Villa_Corpac    36
Name: zona_carabayllo, dtype: int64

In [17]:
df_main['otra_zona_carabayllo'].value_counts()

Alrededores de la Av. Universitaria         12
alrededores de la Av. Universitaria          9
alrededores de av universitaria              8
QATUNA                                       6
ALREDEDORES DE LA AV. UNIVERSITARIA          6
ALREDEDORES DE LA AV UNIVERSITARIA           6
NUEVA IMAGEN                                 4
ALREDEDORES DEL MERCADO NUEVA IMAGEN         3
Alrededores Av universitaria                 3
av universitaria                             2
MERCADO MODELO                               2
Parque 2                                     2
MERCADO QATUNA                               2
Mercado Nueva Imagen                         2
ALREDEDORES DE LA AV SAN FELIPE              1
Alrededores de la Av. universitaria          1
alrededor de av universitaria                1
jiron san francisco carabayllo               1
PARQUE PIMENTEL                              1
ALREDEDORES AV SAN FELIPE                    1
PARQUE FRATERNIDAD                           1
universitaria

In [18]:
df_main['otra_zona_carabayllo'].value_counts().sum()

79

In [19]:
df_main[['mercado_carabayllo', 'otro_mercado_carabayllo']].dropna(how='all').isna().sum()

mercado_carabayllo          0
otro_mercado_carabayllo    21
dtype: int64

In [20]:
df_main['mercado_carabayllo'].value_counts()

otro                 13
Mercado_Tungasuca    12
Mercado_Modelo        9
Name: mercado_carabayllo, dtype: int64

In [21]:
'_1_1_codigo_cuestionario',

'_1_2_fecha_encuesta', 

'_1_3_nombre_encuestador', 


'_1_4_1_ciudad', '_1_4_2_vecindario', 'sector_barrio_huerto', 
'_1_5_perfil_entrevistado',
       
    
    
    
'zona_carabayllo', 'otra_zona_carabayllo', 
'mercado_carabayllo', 'otro_mercado_carabayllo',

'zona_pachacamac', 'otra_zona_pachacamac', 
'mercado_pachacamac', 'otro_mercado_pachacamac',
       
    
'_1_6_ambito_territorial', '_1_7_1_provincia_huerto'
'_1_7_2_canton_huerto', '_1_7_3_parroquia_huerto',
       
    
'_2_1_consentimiento_informado', 
'_3_1_Nombre', 'sexo', 'edad',
'aporta_economicamente_hogar', '_3_2_Informo_telefono', 
'numero_telefono', 'contacto_dirigente', 'contacto_administrador',
'_3_3_1_Distrito_Canton', '_3_3_2_Provincia', '_3_3_3_Pais',
'otro_pais', '_3_4_tiempo_vive_hogar', '_3_4_Vive_vecindario',
'_3_6_nivel_estudio', 'nivel_completo', 'anos_nivel_incompleto'

('_3_6_nivel_estudio', 'nivel_completo', 'anos_nivel_incompleto')

sexo	edad	aporta_economicamente_hogar
num_hombres_hogar	num_menores_hogar	num_miembros_aportan_hogar


In [22]:
df_main.head()

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags
0,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.198,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,172816871,afec3007-3635-415e-b694-bf56ab04a344,2022-07-30 04:13:04,NaN,NaN,submitted_via_web,gaparedes,NaN
1,2,2022-07-29 21:33:38.674,2022-07-30 20:44:17.616,NaN,2022-07-30,Fabian Sevilla,quito,solanda,NaN,productor,...,NaN,NaN,173780648,02a1e1fc-427f-4a40-99fc-ff5237daf6c0,2022-08-04 02:22:08,NaN,NaN,submitted_via_web,fabian1127,NaN
2,3,2022-08-03 14:34:00.904,2022-08-03 15:25:01.405,NaN,2022-08-03,Blanca Minango,quito,solanda,NaN,productor,...,NaN,NaN,173780649,792eaa4e-77e4-4df8-960b-5fb39cf0cd11,2022-08-04 02:22:09,NaN,NaN,submitted_via_web,fabian1127,NaN
3,4,2022-08-05 08:45:32.981,2022-08-05 09:43:39.466,NaN,2022-08-05,Fabian Sevilla,quito,solanda,NaN,productor,...,NaN,NaN,174445889,14040aac-41d2-4224-ae94-ddae03e995d8,2022-08-07 14:50:51,NaN,NaN,submitted_via_web,fabian1127,NaN
4,5,2022-08-05 10:17:44.338,2022-08-05 10:55:58.182,NaN,2022-08-05,Fabian Sevilla,quito,solanda,NaN,productor,...,NaN,NaN,174445895,b4f99bf7-8308-463e-b585-6559835f52bf,2022-08-07 14:50:52,NaN,NaN,submitted_via_web,fabian1127,NaN


## Caracterizacion del encuestado

In [23]:
df_main["_3_2_Informo_telefono"].value_counts()

si    522
no    352
Name: _3_2_Informo_telefono, dtype: int64

### 3.3 Lugar de nacimiento

In [24]:
df_main["_3_3_3_Pais"].value_counts()

peru       481
ecuador    391
otro        15
Name: _3_3_3_Pais, dtype: int64

In [25]:
df_main["_3_3_2_Provincia"].value_counts()

Pichincha                   216
Lima                        176
LIMA                         81
Quito                        27
Cotopaxi                     18
                           ... 
Orellana                      1
Guayaqui                      1
Huanta                        1
Tocache                       1
Santo Domingo de Olleros      1
Name: _3_3_2_Provincia, Length: 178, dtype: int64

In [26]:
df_main["_1_4_1_ciudad"].value_counts().sum()

892

In [27]:
df_main[df_main["_1_4_1_ciudad"].isna()]

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags
186,187,2022-08-22 10:02:09.332,2022-08-22 10:19:02.548,NaN,2022-08-16,Julian Ordoñez,NaN,solanda,NaN,consumidor,...,NaN,NaN,177686949,c3e2305c-e339-44bb-8c67-470f8c72684c,2022-08-22 15:18:04,NaN,NaN,submitted_via_web,funsad,NaN
414,415,2022-08-30 23:42:59.163,2022-08-30 23:43:25.236,NaN,NaT,NaN,NaN,NaN,NaN,consumidor,...,NaN,NaN,180041604,94e0b9d7-3cbe-4fa9-8226-25f9e007a9b3,2022-08-31 04:43:37,NaN,NaN,submitted_via_web,funsad,NaN
884,885,2022-10-19 17:48:15.178,2022-10-29 14:34:46.026,NaN,2022-10-03,Anny Gomez de la Cruz,NaN,pachacamac,NaN,productor,...,NaN,NaN,193631251,0618a2ab-7301-4251-b2e3-0b8c8ffbbce0,2022-10-19 23:33:08,NaN,NaN,submitted_via_web,jorgereyna,NaN


In [28]:
df_main.loc[186, ["_1_4_1_ciudad", "_1_4_2_vecindario"]]

_1_4_1_ciudad            NaN
_1_4_2_vecindario    solanda
Name: 186, dtype: object

In [29]:
df_main.loc[186, "_1_4_1_ciudad"] = "quito"

In [30]:
df_main.loc[186, "_1_4_1_ciudad"]

'quito'

In [31]:
df_main["_1_4_1_ciudad"].isna().sum()

2

In [32]:
df_main.shape

(895, 825)

# Variables Productores
    a_2019_area_produccion
    _1_6_ambito_territorial
    _4_1_principal_actividad_econo
    ingreso_hogar
    _5_Caracteriza_Actividad_Econo
    especificas_productores
    INVERSION_INGRESOS_PRODUCCION
    Porcentaje_Autoconsumo
    _6_6_Dificultades_Comerciales
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_1_Oferta_Alimentos_Produccio
    _7_2_produc_dejo_producir_vend
    _7_5_canal_venta_productores
    _9_2_relaciones_huertos
    aplica_biopreparados
    _10_3_sufrio_lesiones_golpes
    _10_4_sintomas_intoxicacion
    _10_7_productor_problemas_paro
    problemas_paro
    resolucion_problema_paro


# Variables Comerciantes
    mercado_carabayllo
    mercado_pachacamac
    contacto_administrador
    _3_4_Vive_vecindario
    _4_1_principal_actividad_econo
    informacion_puesto_comerciante
    _5_Caracteriza_Actividad_Econo
    especificas_comercianes
    INVERSION_INGRESOS_PRODUCCION
    CAMBIO_INGRESOS
    _6_4_FINANCIA_COSTOS_PUESTO
    _6_5_deudas_del_negocio
    _6_6_Dificultades_Comerciales
    _5_20_participa_organizacion
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_1_Oferta_Alimentos_Produccio
    porcentaje_perdida_semana
    _7_2_produc_dejo_producir_vend
    VENTA_NUEVOS_ALIMENTOS
    _7_4_Actualmente_Usted_utiliza
    _7_6_canal_compra_comerciantes
    _9_2_brindo_apoyo
    _10_2_comerciante_problemas_pa
    problemas_paro
    resolucion_problema_paro


# Variables consumidor
    zona_carabayllo
    zona_pachacamac
    contacto_dirigente
    _3_4_tiempo_vive_hogar
    perdio_empleo
    ingreso_hogar
    participa_organiz_consumidores
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_patrones_compra_consumidores
    _9_2_brindo_apoyo
    Patron_Consumo
    _10_2_consumidor_problemas_par
    problemas_paro
    resolucion_problema_paro
    direccion_hogar



In [33]:
df_main.loc[414].isna().sum()

814

In [34]:
df_main.loc[414][df_main.loc[414].notna()]

_index                                                       415
start                                 2022-08-30 23:42:59.163000
end                                   2022-08-30 23:43:25.236000
_1_5_perfil_entrevistado                              consumidor
__version__                               vwvoSDV79daQYxPddQWKi5
_version_                                 vAyg2NNtmiZULqujb5hCGE
_id                                                    180041604
_uuid                       94e0b9d7-3cbe-4fa9-8226-25f9e007a9b3
_submission_time                             2022-08-31 04:43:37
_status                                        submitted_via_web
_submitted_by                                             funsad
Name: 414, dtype: object

In [35]:
df_main = df_main.drop(414)

In [36]:
df_main[df_main["_1_4_1_ciudad"].isna()]

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags
884,885,2022-10-19 17:48:15.178,2022-10-29 14:34:46.026,NaN,2022-10-03,Anny Gomez de la Cruz,NaN,pachacamac,NaN,productor,...,NaN,NaN,193631251,0618a2ab-7301-4251-b2e3-0b8c8ffbbce0,2022-10-19 23:33:08,NaN,NaN,submitted_via_web,jorgereyna,NaN


In [37]:
df_main.loc[884, ["_1_4_1_ciudad", "_1_4_2_vecindario"]]

_1_4_1_ciudad               NaN
_1_4_2_vecindario    pachacamac
Name: 884, dtype: object

In [38]:
df_main.loc[884, "_1_4_1_ciudad"] = "lima"

In [39]:
for ciudad in df_main["_1_4_1_ciudad"].unique():
    print(ciudad)
    dff = df_main[df_main["_1_4_1_ciudad"] == ciudad]
    unique_values = (dff["_3_3_2_Provincia"]
        .dropna()
        .str.lower()
        .str.strip()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace('[^a-zA-Z ]', '')
        .unique()
    )
    unique_values.sort()
    print(unique_values)

quito
['azuay' 'bolivar' 'canar' 'caracas' 'carchi' 'chimborazo' 'chota'
 'comandancia de barinas' 'cotopaxi' 'cuenca' 'ecuador' 'el oro'
 'esmeraldas' 'guayaqui' 'guayaquil' 'guayas' 'ibarra' 'imbabura' 'loja'
 'los rios' 'manabi' 'manabia' 'manavi' 'morona santiago' 'napo' 'narino'
 'orellana' 'pastaza' 'pichicha' 'pichinca' 'pichincha' 'pillaro'
 'pueblo viejo' 'quito' 'riobamba' 'ruminahui' 'santa elena'
 'santo domingo' 'sto domingo' 'tulcan' 'tumgurahua' 'tunguragua'
 'tungurahua']
lima
['' 'abancay' 'amazonas' 'ancahs' 'ancash' 'andahuaylas' 'angaraes'
 'apurimac' 'arequipa' 'asuncion' 'asunsion' 'asunsionchacas' 'ayacucho'
 'barranca' 'bolivar' 'cajamarca' 'cajatambo' 'cajatambo  lima' 'callao'
 'canchis' 'canete' 'canta' 'carabaya' 'caracas' 'carhuaz' 'celendin'
 'cercado de lima' 'cerro de pasco' 'chachapoyas' 'chachapoyas amazonas'
 'chancay' 'chanchamayo junin' 'chepen' 'chiclayo' 'chota' 'chulcampa'
 'chulucanas' 'chupaca' 'coronel portillo' 'cusco' 'cuzco'
 'departamento 

In [40]:
# Replace after cleaning _3_3_2_Provincia


{ # Ecuador
 'guayaqui': 'guayaquil',
 'pichicha': 'pichincha',
 'pichinca': 'pichincha',
 'sto domingo': 'santo domingo',
 'tumgurahua': 'tungurahua',
 'tunguragua':'tungurahua',
  # Peru
 'ancash': 'ancahs',
    
#     'asuncion' 'asunsion' 'asunsionchacas'
#     'cajatambo' 'cajatambo  lima' 'callao'
#  'canchis' 'canete' 'canta' 'carabaya' 'caracas' 'carhuaz' 'celendin'
#  'cercado de lima' 'cerro de pasco' 
    
#     'ayacucho' 'departamento de ayacucho' 
    
#     'chachapoyas' 'chachapoyas amazonas'
#     'cusco' 'cuzco'
#  'departamento ancash'  
    
#  'huancayo' 'huancayo departamento junin' 
    
#      'huari  ancash'
#      'junin'
#  'tarma departamento junin' 
    
#      'la mar ayacucho' 
#     'lamas  region san martin' 
    
#      'lucana' 'lucanas' 
    
#  'provincia ascope  region la libertad' 
}

{'guayaqui': 'guayaquil',
 'pichicha': 'pichincha',
 'pichinca': 'pichincha',
 'sto domingo': 'santo domingo',
 'tumgurahua': 'tungurahua',
 'tunguragua': 'tungurahua',
 'ancash': 'ancahs'}

In [41]:
for row in df_main["_3_3_2_Provincia"].str.lower().str.strip().value_counts().items():
    print(row)

('lima', 261)
('pichincha', 227)
('quito', 27)
('tungurahua', 18)
('cotopaxi', 18)
('chimborazo', 15)
('piura', 15)
('loja', 14)
('cajamarca', 10)
('ica', 10)
('canta', 9)
('arequipa', 7)
('ancash', 7)
('imbabura', 7)
('cañete', 7)
('bolivar', 7)
('huarochiri', 6)
('huancayo', 6)
('huanuco', 6)
('andahuaylas', 5)
('callao', 4)
('bolívar', 4)
('trujillo', 4)
('ayacucho', 4)
('guayas', 4)
('abancay', 4)
('puno', 4)
('san martin', 3)
('pichincha.', 3)
('junin', 3)
('manabí', 3)
('santo domingo', 3)
('huancavelica', 3)
('cuzco', 3)
('pachacamac', 3)
('caracas', 3)
('carchi', 3)
('el oro', 3)
('manabí.', 2)
('azuay', 2)
('carhuaz', 2)
('huánuco', 2)
('lambayeque', 2)
('cerro de pasco', 2)
('chota', 2)
('junín', 2)
('cañar', 2)
('santa rosa de quives', 2)
('pichinca', 2)
('huaraz', 2)
('huanta', 2)
('lamas - región san martín', 2)
('manabi', 2)
('amazonas', 2)
('los rios', 2)
('huari - ancash', 2)
('iquitos', 2)
('asunsion', 2)
('lucanas', 2)
('pataz', 1)
('puerto cabello', 1)
('apurimac', 1

In [42]:
all_profiles_cols = ['_1_1_codigo_cuestionario',
'_1_2_fecha_encuesta',
'_1_3_nombre_encuestador',
'_1_4_1_ciudad',
'_1_4_2_vecindario',
'sector_barrio_huerto',
'_1_5_perfil_entrevistado',
'_2_1_consentimiento_informado',
'_3_1_Nombre',
'_3_2_Informo_telefono',
'numero_telefono',
'_3_3_1_Distrito_Canton _3_3_2_Provincia _3_3_3_Pais otro_pais',
'_3_6_nivel_estudio',
'nivel_completo',
'anos_nivel_incompleto',
'total_anos_estudio',
'total_miembros_hogar',
'miembro_parentesco',
'otro_pariente',
'otro_NO_pariente',
'sexo',
'edad_miembro',
'miembros_aporta_economicamente',
'perdio_empleo',
'_2019_perdio_empleo',
'_2020_perdio_empleo',
'_2020_tiempo_desempleado',
'_2020_mes_inicio_desempleado',
'_2020_mes_fin_desempleado',
'_2021_perdio_empleo',
'_2021_tiempo_desempleado',
'VARIEDAD',
'_2019_poca_variedad no si ns_n',
'_2019_porque_poca_variedad sin_alimentos sin_recursos',
'_2020_poca_variedad no si ns_nc',
'_2020_porque_poca_variedad sin_alimentos sin_recursos',
'_2021_poca_variedad no si ns_nc',
'_2021_porque_poca_variedad sin_alimentos sin_recursos',
'CANTIDAD',
'_2019_poca_cantidad no si ns_nc',
'_2019_porque_poca_cantidad sin_alimentos sin_recursos',
'_2020_poca_cantidad no si ns_nc',
'_2020_porque_poca_cantidad sin_alimentos sin_recursos',
'_2021_poca_cantidad no si ns_nc',
'_2021_porque_poca_cantidad sin_alimentos sin_recursos',
'SALTO_COMIDA',
'_2019_salto_comida no si ns_nc',
'_2019_porque_salto_comida sin_alimentos sin_recursos',
'_2020_salto_comida no si ns_nc',
'_2020_porque_salto_comida sin_alimentos sin_recursos',
'_2021_salto_comida no si ns_nc',
'_2021_porque_salto_comida sin_alimentos sin_recursos',
'NO_COMIO_1_DIA',
'_2019_no_comio_1_dia no si ns_nc',
'_2019_porque_no_comio_1_dia sin_alimentos sin_recursos',
'_2020_no_comio_1_dia no si ns_nc',
'_2020_porque_no_comio_1_dia sin_alimentos sin_recursos',
'_2021_no_comio_1_dia no si ns_nc',
'_2021_porque_no_comio_1_dia sin_alimentos sin_recursos',
'_8_5_medidas_acceso_alimentos',
'_1er_opcion / _2da_opcion / _3er_opcion / _4ta_opcion / _5ta_opcion / _6ta_opcion / _7ma_opcion / _8va_opcion / _9na_opcion / _10ma_opcion pudo_comprar_alimentos uso_ahorros busco_trabajo saco_negocio contacto_organizacion contacto_productores_comercian redes_delivery intercambio_alimentos recibio_alimentos otro',
'otra_medida_para_tener_aliment',
'_9_1_recibio_apoyo no si',
'_a_de_quien_recibio_apoyo personas grupos organizaciones',
'personas_recibio_apoyo familia amigo vecino trabajo otra_persona',
'otras_personas_recibio_apoyo',
'parentesco_recibio_apoyo esposos hijos_hijastros yerno_nuera nietos padres suegros hermanos otro_pariente',
'otros_parientes_recibio_apoyo',
'grupo_recibio_apoyo',
'organizacion_recibio_apoyo vecinal ong iglesia empresa cooperacion centros_educacion gobierno otra_organizacion',
'_b_Sexo_recibio_apoyo hombre mujer',
'Nombre_grupo_organiz_recibio',
'_c_vecindario_recibio_apoyo si no',
'_d_tipo_apoyo_recibido dinero alimentos informaciones otro_apoyo',
'otro_tipo_apoyo_recibido',
'_e_frecuencia_recibio_apoyo una_vez algunas_veces frecuentemente',
'_f_Actualmente_recibe_apoyo si no',
'_10_1_padece_problemas_salud presion_alta diabetes sobrepeso bajo_peso otros_problemas no',
'otros_problemas_salud',
'problemas_paro',
'resolucion_problema_paro',
'datos_geo_referenciados',
'_datos_geo_referenciados_latitude',
'_datos_geo_referenciados_longitude',
'_datos_geo_referenciados_altitude',
'_datos_geo_referenciados_precision',
'Incluir_datos_georre_ciados_LATITUD_UTM',
'Incluir_datos_georre_iados_LONGITUD_UTM',]


In [43]:
len(all_profiles_cols)

88

In [44]:
'_1_1_codigo_cuestionario',
'_1_2_fecha_encuesta',
'_1_3_nombre_encuestador',
'_1_4_1_ciudad',
'_1_4_2_vecindario',
'sector_barrio_huerto',
'_1_5_perfil_entrevistado',

('_1_5_perfil_entrevistado',)

In [45]:
df_main['sector_barrio_huerto']

0            Tolontag Centro
1                        NaN
2                        NaN
3                        NaN
4                        NaN
               ...          
890    Asociación 24 de mayo
891        Penileo - Píllaro
892                      NaN
893                  no puso
894              Cieneguilla
Name: sector_barrio_huerto, Length: 894, dtype: object

In [46]:
df_main['_1_4_2_vecindario'].isna().sum()

0

In [47]:
df_main['_1_4_2_vecindario'].unique()

array(['fuera', 'solanda', 'la_magdalena', 'carabayllo', 'pachacamac'],
      dtype=object)

In [48]:
df_main['_1_4_2_vecindario'].value_counts()

carabayllo      273
solanda         217
pachacamac      213
la_magdalena    159
fuera            32
Name: _1_4_2_vecindario, dtype: int64

In [49]:
df_main['sector_barrio_huerto'].sort_values().unique()

array(['Aso. 24 de mayo - Cusubamba', 'Asociación 24 de mayo',
       'Campo Alegre - Yachil', 'Churagpamba', 'Cieneguilla',
       'Cocha Verde', 'Huapante - Pillaro', 'Huapante - San Andrés',
       'Huapante Chico - Píllaro', 'Huapante chico', 'La Argelia',
       'Penileo', 'Penileo - Píllaro', 'Quillan - La Planta', 'Saguangal',
       'San Andrés', 'San Pedro de Cusbamba', 'San Pedro de Cusubamba',
       'San Vicente de Pillaro',
       'Sector 1 de Canarias Santa Rosa de Pingulmí', 'Tolontag Centro',
       'Tolontag-Pintag', 'Yatchil', 'antioquia', 'cieneguilla', 'lurin',
       'no puso', nan], dtype=object)

In [50]:
df_main['sector_barrio_huerto'].notna().sum()

31

In [51]:
df_main['sector_barrio_huerto'].value_counts()

Penileo - Píllaro                              3
Yatchil                                        2
lurin                                          2
Tolontag Centro                                1
no puso                                        1
Asociación 24 de mayo                          1
Aso. 24 de mayo - Cusubamba                    1
Sector 1 de Canarias Santa Rosa de Pingulmí    1
Huapante Chico - Píllaro                       1
antioquia                                      1
cieneguilla                                    1
Huapante - San Andrés                          1
Quillan - La Planta                            1
Campo Alegre - Yachil                          1
Cocha Verde                                    1
Saguangal                                      1
Huapante chico                                 1
Huapante - Pillaro                             1
San Andrés                                     1
Penileo                                        1
Churagpamba         

In [52]:
def expand_dict(dictionary):
    """
    Expand {k: [v0, v1, v2, ...], ...} in dict to  
    {v0: k, v1: k, v2: k, ...} 
    """
    return {el: k for k, v in dictionary.items() for el in v}

In [53]:
expand_dict({
    'Cusubamba': ['Aso. 24 de mayo - Cusubamba', 'Asociación 24 de mayo', 'San Pedro de Cusbamba', 'San Pedro de Cusubamba'],
    'Huapante': ['Huapante - Pillaro', 'Huapante - San Andrés', 'Huapante Chico - Píllaro', 'Huapante chico'],
    'Tolontag': ['Tolontag Centro', 'Tolontag-Pintag'],
    'Cieneguilla': ['cieneguilla'],
    'Yatchil': ['Campo Alegre - Yachil', 'Yatchil'],
    'Penileo': ['Penileo - Píllaro'],
    'no puso': [np.nan]
})

{'Aso. 24 de mayo - Cusubamba': 'Cusubamba',
 'Asociación 24 de mayo': 'Cusubamba',
 'San Pedro de Cusbamba': 'Cusubamba',
 'San Pedro de Cusubamba': 'Cusubamba',
 'Huapante - Pillaro': 'Huapante',
 'Huapante - San Andrés': 'Huapante',
 'Huapante Chico - Píllaro': 'Huapante',
 'Huapante chico': 'Huapante',
 'Tolontag Centro': 'Tolontag',
 'Tolontag-Pintag': 'Tolontag',
 'cieneguilla': 'Cieneguilla',
 'Campo Alegre - Yachil': 'Yatchil',
 'Yatchil': 'Yatchil',
 'Penileo - Píllaro': 'Penileo',
 nan: 'no puso'}

In [54]:
df_main['sector_barrio_huerto'].replace(expand_dict({
    'Cusubamba': ['Aso. 24 de mayo - Cusubamba', 'Asociación 24 de mayo', 'San Pedro de Cusbamba', 'San Pedro de Cusubamba'],
    'Huapante': ['Huapante - Pillaro', 'Huapante - San Andrés', 'Huapante Chico - Píllaro', 'Huapante chico'],
    'Tolontag': ['Tolontag Centro', 'Tolontag-Pintag'],
    'Cieneguilla': ['cieneguilla'],
    'Yatchil': ['Campo Alegre - Yachil', 'Yatchil'],
    'Penileo': ['Penileo - Píllaro'],
    np.nan: ['no puso']
}), inplace=True)

In [55]:
df_main.loc[df_main['_1_4_2_vecindario'] != 'fuera', 'sector_barrio_huerto'] = 'No aplica'

In [56]:
df_main['sector_barrio_huerto'].value_counts()

No aplica                                      862
Cusubamba                                        4
Penileo                                          4
Huapante                                         4
Yatchil                                          3
Tolontag                                         2
lurin                                            2
Cieneguilla                                      2
antioquia                                        1
Quillan - La Planta                              1
San Andrés                                       1
Cocha Verde                                      1
Churagpamba                                      1
San Vicente de Pillaro                           1
La Argelia                                       1
Saguangal                                        1
Sector 1 de Canarias Santa Rosa de Pingulmí      1
Name: sector_barrio_huerto, dtype: int64

In [57]:
['_3_1_Nombre', '_3_2_Informo_telefono', 'numero_telefono', '_3_3_1_Distrito_Canton',
 '_3_3_2_Provincia _3_3_3_Pais otro_pais', '_3_6_nivel_estudio', 'nivel_completo',
 'anos_nivel_incompleto', 'total_anos_estudio', 'total_miembros_hogar', 'miembro_parentesco',
 'otro_pariente', 'otro_NO_pariente', 'sexo', 'edad_miembro', 'miembros_aporta_economicamente']

['_3_1_Nombre',
 '_3_2_Informo_telefono',
 'numero_telefono',
 '_3_3_1_Distrito_Canton',
 '_3_3_2_Provincia _3_3_3_Pais otro_pais',
 '_3_6_nivel_estudio',
 'nivel_completo',
 'anos_nivel_incompleto',
 'total_anos_estudio',
 'total_miembros_hogar',
 'miembro_parentesco',
 'otro_pariente',
 'otro_NO_pariente',
 'sexo',
 'edad_miembro',
 'miembros_aporta_economicamente']

In [58]:
df_main['_3_1_Nombre'] = df_main['_3_1_Nombre'].str.title()

In [59]:
df_main['_3_2_Informo_telefono'].unique()

array(['si', 'no', nan], dtype=object)

In [60]:
df_main['_3_2_Informo_telefono'].isna().sum()

20

In [61]:
df_main[df_main['_3_2_Informo_telefono'].isna()]['numero_telefono']

62    NaN
105   NaN
258   NaN
264   NaN
266   NaN
280   NaN
313   NaN
354   NaN
391   NaN
453   NaN
497   NaN
500   NaN
701   NaN
731   NaN
767   NaN
768   NaN
769   NaN
792   NaN
793   NaN
817   NaN
Name: numero_telefono, dtype: float64

In [62]:
df_main.loc[df_main['_3_2_Informo_telefono'].isna(), '_3_2_Informo_telefono'] = 'no'

In [63]:
df_main['_3_2_Informo_telefono'].unique()

array(['si', 'no'], dtype=object)

In [64]:
df_main['_3_2_Informo_telefono'].value_counts()

si    522
no    372
Name: _3_2_Informo_telefono, dtype: int64

In [65]:
df_main['numero_telefono'].isna().sum()

374

In [66]:
df_main['numero_telefono_clean'] = (df_main['numero_telefono']
    .convert_dtypes(convert_integer=True)
    .astype(str).replace('<NA>', np.nan))

In [67]:
df_main['numero_telefono_clean'].isna().sum()

374

In [68]:
df_main[df_main['_3_2_Informo_telefono'] == 'si']['numero_telefono'].isna().sum()

2

In [69]:
df_main.loc[
    (df_main['_3_2_Informo_telefono'] == 'si') & (df_main['numero_telefono'].isna()),   
    '_3_2_Informo_telefono'
] = 'no'

In [70]:
df_main['_3_2_Informo_telefono'].value_counts()

si    520
no    374
Name: _3_2_Informo_telefono, dtype: int64

In [71]:
df_main[df_main['_3_2_Informo_telefono'] == 'no']['numero_telefono'].isna().sum()

374

In [72]:
df_main.loc[df_main['_3_2_Informo_telefono'] == 'no', 'numero_telefono'] = 'No informó'

In [73]:
for val in df_main['_3_2_Informo_telefono'].unique():
    print(val)
    print(df_main[df_main['_3_2_Informo_telefono'] == val]['numero_telefono'].value_counts())
    print()

si
9.690606e+08    2
9.812219e+08    2
9.870629e+08    2
9.925683e+08    2
9.141059e+08    2
               ..
9.172401e+09    1
9.027669e+08    1
9.388013e+08    1
9.927099e+09    1
9.889283e+08    1
Name: numero_telefono, Length: 509, dtype: int64

no
No informó    374
Name: numero_telefono, dtype: int64



In [ ]:
'_3_3_1_Distrito_Canton',
 '_3_3_2_Provincia',
    

In [81]:
df_main['_3_3_3_Pais'].unique()

array(['ecuador', 'peru', nan, 'otro'], dtype=object)

In [85]:
df_main['_3_3_3_Pais'].isna().sum()

7

In [87]:
df_main['_3_3_3_Pais'].value_counts()

peru       481
ecuador    391
otro        15
Name: _3_3_3_Pais, dtype: int64

In [86]:
df_main[df_main['_3_3_3_Pais'].isna()]['otro_pais']

34     NaN
147    NaN
182    NaN
233    NaN
273    NaN
445    NaN
514    NaN
Name: otro_pais, dtype: object

In [88]:
df_main['_3_3_3_Pais'] = df_main['_3_3_3_Pais'].fillna('No informó')

In [90]:
df_main['_3_3_3_Pais'].value_counts()

peru          481
ecuador       391
otro           15
No informó      7
Name: _3_3_3_Pais, dtype: int64

In [92]:
df_main['_3_3_3_Pais'].unique()

array(['ecuador', 'peru', 'No informó', 'otro'], dtype=object)

In [80]:
df_main['otro_pais'].unique()

array([nan, 'Colombia', 'Venezuela', 'México', 'Venezuela.', 'Francia',
       'Alemania', 'Brasil', 'VENEZUELA'], dtype=object)

In [82]:
df_main['otro_pais'].value_counts()

Venezuela     4
VENEZUELA     4
Colombia      2
México        1
Venezuela.    1
Francia       1
Alemania      1
Brasil        1
Name: otro_pais, dtype: int64

In [84]:
df_main['otro_pais'].notna().sum()

15

In [ ]:
df_main['pais_clean'] = df_main['_3_3_3_Pais'] == 'otro'

In [79]:
df_main['_3_4_3_Pa_s'].unique()

array([nan])

In [ ]:
('lima', 261)
('pichincha', 227)
('quito', 27)
('tungurahua', 18)
('cotopaxi', 18)
('chimborazo', 15)
('piura', 15)
('loja', 14)
('cajamarca', 10)
('ica', 10)
('canta', 9)
('arequipa', 7)
('ancash', 7)
('imbabura', 7)
('cañete', 7)
('bolivar', 7)
('huarochiri', 6)
('huancayo', 6)
('huanuco', 6)
('andahuaylas', 5)
('callao', 4)
('bolívar', 4)
('trujillo', 4)
('ayacucho', 4)
('guayas', 4)
('abancay', 4)
('puno', 4)
('san martin', 3)
('pichincha.', 3)
('junin', 3)
('manabí', 3)
('santo domingo', 3)
('huancavelica', 3)
('cuzco', 3)
('pachacamac', 3)
('caracas', 3)
('carchi', 3)
('el oro', 3)
('manabí.', 2)
('azuay', 2)
('carhuaz', 2)
('huánuco', 2)
('lambayeque', 2)
('cerro de pasco', 2)
('chota', 2)
('junín', 2)
('cañar', 2)
('santa rosa de quives', 2)
('pichinca', 2)
('huaraz', 2)
('huanta', 2)
('lamas - región san martín', 2)
('manabi', 2)
('amazonas', 2)
('los rios', 2)
('huari - ancash', 2)
('iquitos', 2)
('asunsion', 2)
('lucanas', 2)
('pataz', 1)
('puerto cabello', 1)
('apurimac', 1)
('tulcán', 1)
('mariscal caceres', 1)
('rumiñahui', 1)
('pasco', 1)
('morropon', 1)
('asuncion', 1)
('san marcos', 1)
('portuguesa', 1)
('madre de dios', 1)
('riobamba', 1)
('barranca', 1)
('chulucanas', 1)
('chachapoyas', 1)
('nariño', 1)
('-', 1)
('loreto', 1)
('chepen', 1)
('lucana', 1)
('tunguragua', 1)
('celendín', 1)
('asunsion-chacas', 1)
('tarma', 1)
('cusco', 1)
('chupaca', 1)
('chachapoyas -amazonas', 1)
('la mar- ayacucho', 1)
('la libertad', 1)
('angaraes', 1)
('yaracuy', 1)
('pillaro', 1)
('chulcampa', 1)
('cercado de lima', 1)
('canchis', 1)
('recuay', 1)
('maynas', 1)
('melgar', 1)
('orellana', 1)
('chancay', 1)
('los ríos', 1)
('sandia', 1)
('guayaquil', 1)
('ibarra', 1)
('pueblo viejo', 1)
('tarma, departamento junín', 1)
('cajatambo - lima', 1)
('cuenca', 1)
('provincia ascope - región la libertad', 1)
('los ríos.', 1)
('napo', 1)
('coronel portillo', 1)
('sto domingo.', 1)
('paruro', 1)
('huaral', 1)
('pallasca', 1)
('chiclayo', 1)
('comandancia de barinas', 1)
('santa elena', 1)
('ancahs', 1)
('pichicha.', 1)
('departamento ancash', 1)
('esmeraldas.', 1)
('santa', 1)
('pastaza', 1)
('tacna', 1)
('essen', 1)
('yungay', 1)
('carabaya', 1)
('lampa', 1)
('panao- huánuco', 1)
('huaro', 1)
('huancayo, departamento junín', 1)
('chanchamayo (junin)', 1)
('manavi', 1)
('imbabura.', 1)
('manabía', 1)
('pichicha', 1)
('ecuador', 1)
('tumgurahua', 1)
('morona santiago', 1)
('departamento de ayacucho', 1)
('cajatambo', 1)
('guayaqui', 1)
('tocache', 1)
('santo domingo de olleros', 1)

In [ ]:
df_main["_3_3_1_Distrito_Canton"].value_counts()